# Importing Libraries

In [18]:
from tkinter import *
import os
# import Image
import ctypes
import PIL
from PIL import ImageTk
from PIL import Image
from PIL import ImageOps
from tkinter.filedialog import*
import tkinter.messagebox
import imghdr
from PIL import ImageDraw
from collections import*

import numpy as np
import cv2
import pytesseract
import pyttsx3

import winsound
from scipy.io.wavfile import read, write
from IPython.display import Audio
import googletrans
from googletrans import Translator
from gtts import gTTS 

  

# Crop Image

In [19]:
def crop(canvas):
    canvas.data.colourPopToHappen=False
    canvas.data.drawOn=False
    # have to check if crop button is pressed or not, otherwise,
    # the root events which point to
    # different functions based on what button has been pressed
    # will get mixed up 
    canvas.data.cropPopToHappen=True
    tkinter.messagebox.showinfo(title="Crop", \
                          message="Draw cropping rectangle and press Enter" ,\
                          parent=canvas.data.mainWindow)
    if canvas.data.image!=None:
        canvas.data.mainWindow.bind("<ButtonPress-1>", \
                                    lambda event: startCrop(event, canvas))
        canvas.data.mainWindow.bind("<B1-Motion>",\
                                    lambda event: drawCrop(event, canvas))
        canvas.data.mainWindow.bind("<ButtonRelease-1>", \
                                    lambda event: endCrop(event, canvas))

def startCrop(event, canvas):
    # detects the start of the crop rectangle
    if canvas.data.endCrop==False and canvas.data.cropPopToHappen==True:
        canvas.data.startCropX=event.x
        canvas.data.startCropY=event.y

def drawCrop(event,canvas):
    # keeps extending the crop rectange as the user extends
    # his desired crop rectangle
    if canvas.data.endCrop==False and canvas.data.cropPopToHappen==True:
        canvas.data.tempCropX=event.x
        canvas.data.tempCropY=event.y
        canvas.create_rectangle(canvas.data.startCropX, \
                                canvas.data.startCropY,
                                 canvas.data.tempCropX, \
            canvas.data.tempCropY, fill="gray", stipple="gray12", width=0)

def endCrop(event, canvas):
    # set canvas.data.endCrop=True so that button pressed movements
    # are not caught anymore but set it to False when "Enter"
    # is pressed so that crop can be performed another time too
    if canvas.data.cropPopToHappen==True:
        canvas.data.endCrop=True
        canvas.data.endCropX=event.x
        canvas.data.endCropY=event.y
        canvas.create_rectangle(canvas.data.startCropX, \
                                canvas.data.startCropY,
                                 canvas.data.endCropX, \
            canvas.data.endCropY, fill="gray", stipple="gray12", width=0 )
        canvas.data.mainWindow.bind("<Return>", \
                                lambda event: performCrop(event, canvas))

def performCrop(event,canvas):
    canvas.data.image=\
    canvas.data.image.crop(\
    (int(round((canvas.data.startCropX-canvas.data.imageTopX)*canvas.data.imageScale)),
    int(round((canvas.data.startCropY-canvas.data.imageTopY)*canvas.data.imageScale)),
    int(round((canvas.data.endCropX-canvas.data.imageTopX)*canvas.data.imageScale)),
    int(round((canvas.data.endCropY-canvas.data.imageTopY)*canvas.data.imageScale))))
    canvas.data.endCrop=False
    canvas.data.cropPopToHappen=False
    save(canvas)
    canvas.data.undoQueue.append(canvas.data.image.copy())
    canvas.data.imageForTk=makeImageForTk(canvas)
    drawImage(canvas)
    
    

# Rotate Image

In [20]:
def rotateFinished(canvas, rotateWindow, rotateSlider, previousAngle):
    if canvas.data.rotateWindowClose==True:
        rotateWindow.destroy()
        canvas.data.rotateWindowClose=False
    else:
        if canvas.data.image!=None and rotateWindow.winfo_exists():
            canvas.data.angleSelected=rotateSlider.get()
            if canvas.data.angleSelected!= None and \
               canvas.data.angleSelected!= previousAngle:
                canvas.data.image=\
                canvas.data.image.rotate(float(canvas.data.angleSelected))
                canvas.data.imageForTk=makeImageForTk(canvas)
                drawImage(canvas)
        canvas.after(200, lambda:rotateFinished(canvas,\
                    rotateWindow, rotateSlider, canvas.data.angleSelected) )


def closeRotateWindow(canvas):
    if canvas.data.image!=None:
        save(canvas)
        canvas.data.undoQueue.append(canvas.data.image.copy())
        canvas.data.rotateWindowClose=True
    
def rotate(canvas):
    canvas.data.colourPopToHappen=False
    canvas.data.cropPopToHappen=False
    canvas.data.drawOn=False
    rotateWindow=Toplevel(canvas.data.mainWindow)
    rotateWindow.title("Rotate")
    rotateSlider=Scale(rotateWindow, from_=0, to=360, orient=HORIZONTAL)
    rotateSlider.pack()
    OkRotateFrame=Frame(rotateWindow)
    OkRotateButton=Button(OkRotateFrame, text="OK",\
                          command=lambda: closeRotateWindow(canvas))
    OkRotateButton.grid(row=0,column=0)
    OkRotateFrame.pack(side=BOTTOM)
    rotateFinished(canvas, rotateWindow, rotateSlider, 0)


# Brightness of Image

In [21]:
def closeBrightnessWindow(canvas):
    if canvas.data.image!=None:
        save(canvas)
        canvas.data.undoQueue.append(canvas.data.image.copy())
        canvas.data.brightnessWindowClose=True

def changeBrightness(canvas, brightnessWindow, brightnessSlider, \
                     previousVal):
    if canvas.data.brightnessWindowClose==True:
        brightnessWindow.destroy()
        canvas.data.brightnessWindowClose=False
        
    else:
        # increasing pixel values according to slider value increases
        #brightness we change ot according to the difference between the
        # previous value and the current slider value
        if canvas.data.image!=None and brightnessWindow.winfo_exists():
            sliderVal=brightnessSlider.get()
            scale=(sliderVal-previousVal)/100.0
            canvas.data.image=canvas.data.image.point(\
                lambda i: i+ int(round(i*scale)))  
            canvas.data.imageForTk=makeImageForTk(canvas)
            drawImage(canvas)
            canvas.after(200, \
            lambda: changeBrightness(canvas, brightnessWindow, \
                                     brightnessSlider, sliderVal))

def brightness(canvas):
    canvas.data.colourPopToHappen=False
    canvas.data.cropPopToHappen=False
    canvas.data.drawOn=False
    brightnessWindow=Toplevel(canvas.data.mainWindow)
    brightnessWindow.title("Brightness")
    brightnessSlider=Scale(brightnessWindow, from_=-100, to=100,\
                           orient=HORIZONTAL)
    brightnessSlider.pack()
    OkBrightnessFrame=Frame(brightnessWindow)
    OkBrightnessButton=Button(OkBrightnessFrame, text="OK", \
                              command=lambda: closeBrightnessWindow(canvas))
    OkBrightnessButton.grid(row=0,column=0)
    OkBrightnessFrame.pack(side=BOTTOM)
    changeBrightness(canvas, brightnessWindow, brightnessSlider,0)
    brightnessSlider.set(0)


# Mirror Image

In [22]:
def mirror(canvas):
    canvas.data.colourPopToHappen=False
    canvas.data.cropPopToHappen=False
    canvas.data.drawOn=False
    if canvas.data.image!=None:
        canvas.data.image=ImageOps.mirror(canvas.data.image)
        save(canvas)
        canvas.data.undoQueue.append(canvas.data.image.copy())
        canvas.data.imageForTk=makeImageForTk(canvas)
        drawImage(canvas)

# Flip Image

In [23]:
def flip(canvas):
    canvas.data.colourPopToHappen=False
    canvas.data.cropPopToHappen=False
    canvas.data.drawOn=False
    if canvas.data.image!=None:
        canvas.data.image=ImageOps.flip(canvas.data.image)
        save(canvas)
        canvas.data.undoQueue.append(canvas.data.image.copy())
        canvas.data.imageForTk=makeImageForTk(canvas)
        drawImage(canvas)



# Reset Image

In [24]:
def reset(canvas):
    canvas.data.colourPopToHappen=False
    canvas.data.cropPopToHappen=False
    canvas.data.drawOn=False
    ### change back to original image
    if canvas.data.image!=None:
        canvas.data.image=canvas.data.originalImage.copy()
        save(canvas)
        canvas.data.undoQueue.append(canvas.data.image.copy())
        canvas.data.imageForTk=makeImageForTk(canvas)
        drawImage(canvas)

# Menu Bar

# Open New File (Image)

In [25]:
def newImage(canvas):
    imageName=askopenfilename()
    filetype=""
    #make sure it's an image file
    try: filetype=imghdr.what(imageName)
    except:
        tkinter.messagebox.showinfo(title="Image File",\
        message="Choose an Image File!" , parent=canvas.data.mainWindow)
    # restrict filetypes to .jpg, .bmp, etc.
    if filetype in ['jpg','jpeg', 'bmp', 'png']:
        canvas.data.imageLocation=imageName
        im = PIL.Image.open(imageName)
        
        canvas.data.image=im
        canvas.data.originalImage=im.copy()
        canvas.data.undoQueue.append(im.copy())
        canvas.data.imageSize=im.size #Original Image dimensions
        canvas.data.imageForTk=makeImageForTk(canvas)
        drawImage(canvas)
    else:
        tkinter.messagebox.showinfo(title="Image File",\
        message="Choose an Image File!" , parent=canvas.data.mainWindow)

# Save

In [26]:
def save(canvas):
    if canvas.data.image!=None:
        im=canvas.data.image
        im.save(canvas.data.imageLocation)

# Save As

In [27]:
def saveAs(canvas):
    # ask where the user wants to save the file
    if canvas.data.image!=None:
        filename=asksaveasfilename(defaultextension=".jpg")
        im=canvas.data.image
        im.save(filename)

# Edit

In [28]:
def keyPressed(canvas, event):
    if event.keysym=="z":
        undo(canvas)
    elif event.keysym=="y":
        redo(canvas)
        

# we use deques so as to make Undo and Redo more efficient and avoid
# memory space isuues 
# after each change, we append the new version of the image to
# the Undo queue
def undo(canvas):
    if len(canvas.data.undoQueue)>0:
        # the last element of the Undo Deque is the
        # current version of the image
        lastImage=canvas.data.undoQueue.pop()
        # we would want the current version if wehit redo after undo
        canvas.data.redoQueue.appendleft(lastImage)
    if len(canvas.data.undoQueue)>0:
        # the previous version of the image
        canvas.data.image=canvas.data.undoQueue[-1]
    save(canvas)
    canvas.data.imageForTk=makeImageForTk(canvas)
    drawImage(canvas)

def redo(canvas):
    if len(canvas.data.redoQueue)>0:
        canvas.data.image=canvas.data.redoQueue[0]
    save(canvas)
    if len(canvas.data.redoQueue)>0:
        # we remove this version from the Redo Deque beacuase it
        # has become our current image
        lastImage=canvas.data.redoQueue.popleft()
        canvas.data.undoQueue.append(lastImage)
    canvas.data.imageForTk=makeImageForTk(canvas)
    drawImage(canvas)

# Filters

In [29]:
def invert(canvas):
    canvas.data.colourPopToHappen=False
    canvas.data.cropPopToHappen=False
    canvas.data.drawOn=False
    if canvas.data.image!=None:
        canvas.data.image=ImageOps.invert(canvas.data.image)
        save(canvas)
        canvas.data.undoQueue.append(canvas.data.image.copy())
        canvas.data.imageForTk=makeImageForTk(canvas)
        drawImage(canvas)

def solarize(canvas):
    canvas.data.colourPopToHappen=False
    canvas.data.cropPopToHappen=False
    solarizeWindow=Toplevel(canvas.data.mainWindow)
    solarizeWindow.title("Solarize")
    solarizeSlider=Scale(solarizeWindow, from_=0, to=255, orient=HORIZONTAL)
    solarizeSlider.pack()
    OkSolarizeFrame=Frame(solarizeWindow)
    OkSolarizeButton=Button(OkSolarizeFrame, text="OK",\
                            command=lambda: closeSolarizeWindow(canvas))
    OkSolarizeButton.grid(row=0,column=0)
    OkSolarizeFrame.pack(side=BOTTOM)
    ### beacsue intial silderVal=0
    performSolarize(canvas, solarizeWindow, solarizeSlider, 255)
    

def performSolarize(canvas, solarizeWindow, solarizeSlider, previousThreshold):
    if canvas.data.solarizeWindowClose==True:
        solarizeWindow.destroy()
        canvas.data.solarizeWindowClose=False
        
    else:
        # the  slider denotes the % of solarization thta the user wants,
        # so the threshold (above which pixels are inverted) is inversely
        # related to the slider value
        if solarizeWindow.winfo_exists():
            sliderVal=solarizeSlider.get()
            threshold_=255-sliderVal
            if canvas.data.image!=None and threshold_!=previousThreshold:
                canvas.data.image=ImageOps.solarize(canvas.data.image,\
                                                    threshold=threshold_)
                canvas.data.imageForTk=makeImageForTk(canvas)
                drawImage(canvas)
            canvas.after(200, lambda: performSolarize(canvas, \
                                solarizeWindow, solarizeSlider, threshold_))

def closeSolarizeWindow(canvas):
    if canvas.data.image!=None:
        save(canvas)
        canvas.data.undoQueue.append(canvas.data.image.copy())
        canvas.data.solarizeWindowClose=True
    

# Image Displaying For Tk

In [30]:
def makeImageForTk(canvas):
    im=canvas.data.image
    if canvas.data.image!=None:
        # Beacuse after cropping the now 'image' might have diffrent
        # dimensional ratios
        imageWidth=canvas.data.image.size[0] 
        imageHeight=canvas.data.image.size[1]
        #To make biggest version of the image fit inside the canvas
        if imageWidth>imageHeight:
            resizedImage=im.resize((canvas.data.width,\
                int(round(float(imageHeight)*canvas.data.width/imageWidth))))
            # store the scale so as to use it later
            canvas.data.imageScale=float(imageWidth)/canvas.data.width
        else:
            resizedImage=im.resize((int(round(float(imageWidth)*canvas.data.height/imageHeight)),\
                                    canvas.data.height))
            canvas.data.imageScale=float(imageHeight)/canvas.data.height
        # we may need to refer to ther resized image atttributes again
        canvas.data.resizedIm = resizedImage
      
        return ImageTk.PhotoImage(resizedImage)

def drawImage(canvas):
    if canvas.data.image!=None:
        # make the canvas center and the image center the same
        canvas.create_image(canvas.data.width/2.0-canvas.data.resizedIm.size[0]/2.0,
                        canvas.data.height/2.0-canvas.data.resizedIm.size[1]/2.0,
                            anchor=NW, image=canvas.data.imageForTk)
        canvas.data.imageTopX=int(round(canvas.data.width/2.0-canvas.data.resizedIm.size[0]/2.0))
        canvas.data.imageTopY=int(round(canvas.data.height/2.0-canvas.data.resizedIm.size[1]/2.0))



# Scan Image

In [31]:
def scan(imageName):
    #images=canvas.data.image
    #cv2.imshow('',images)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()
    #read image
    img = cv2.imread(imageName,1)
    #resize image
    img = np.array(img)
    img = cv2.resize(img,(600,800))
    #convert image to grayscale
    grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #blurr image to smooth 
    blurr = cv2.GaussianBlur(grey, (5,5),0)
    #finding edges 
    edge = cv2.Canny(blurr, 0, 50)   

#apadtive threshold and canny gave similar final output 
#threshold = cv2.adaptiveThreshold(blurr ,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,11,2)
#find contours in thresholded image and sort them according to decreasing area
    contours, _ = cv2.findContours(edge, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=cv2.contourArea, reverse= True)

    #contour approximation 
    for i in contours:
        elip =  cv2.arcLength(i, True)
        approx = cv2.approxPolyDP(i,0.08*elip, True)

        if len(approx) == 4 : 
            doc = approx 
            break
        else:
            print("no contour found")

    #draw contours 
    cv2.drawContours(img, [doc], -1, (0, 255, 0), 2)

    #reshape to avoid errors ahead
    doc=doc.reshape((4,2))

    #create a new array and initialize 
    new_doc = np.zeros((4,2), dtype="float32")

    Sum = doc.sum(axis = 1)
    new_doc[0] = doc[np.argmin(Sum)]
    new_doc[2] = doc[np.argmax(Sum)]

    Diff = np.diff(doc, axis=1)
    new_doc[1] = doc[np.argmin(Diff)]
    new_doc[3] = doc[np.argmax(Diff)]

    (tl,tr,br,bl) = new_doc

    #find distance between points and get max 
    dist1 = np.linalg.norm(br-bl)
    dist2 = np.linalg.norm(tr-tl)

    maxLen = max(int(dist1),int(dist2))

    dist3 = np.linalg.norm(tr-br)
    dist4 = np.linalg.norm(tl-bl)

    maxHeight = max(int(dist3), int(dist4))

    dst = np.array([[0,0],[maxLen-1, 0],[maxLen-1, maxHeight-1], [0, maxHeight-1]], dtype="float32")

    N = cv2.getPerspectiveTransform(new_doc, dst)
    warp = cv2.warpPerspective(img, N, (maxLen, maxHeight))

    img2 = cv2.cvtColor(warp, cv2.COLOR_BGR2GRAY)
    img2 = cv2.resize(img2,(600,800))
    cv2.imshow("Scanned.jpg", img2)

    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return img2
    
def imageScanner(canvas):
    canvas.data.colourPopToHappen=False
    canvas.data.cropPopToHappen=False
    canvas.data.drawOn=False
    if canvas.data.image!=None:
        imageName=askopenfilename()
        filetype=""
        #make sure it's an image file
        try: filetype=imghdr.what(imageName)
        except:
            tkinter.messagebox.showinfo(title="Image File",message="Choose an Image File!" , parent=canvas.data.mainWindow)
        # restrict filetypes to .jpg, .bmp, etc.
        if filetype in ['jpg','jpeg', 'bmp', 'png']:
            cleaned_image = scan(imageName)
        else:
            tkinter.messagebox.showinfo(title="Image File",\
            message="Choose an Image File!" , parent=canvas.data.mainWindow)
        #cleaned_image = scan(canvas)
        Final_Image=cv2.imwrite("scanned_output.png", cleaned_image)

# Extract Text from Image

In [32]:
def image_to_text(canvas):
    pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
    text=pytesseract.image_to_string(canvas.data.image)
    return text

def extract_text(canvas):
    canvas.data.colourPopToHappen=False
    canvas.data.cropPopToHappen=False
    canvas.data.drawOn=False
    if canvas.data.image!=None:
        LanguageWindow=Toplevel(canvas.data.mainWindow)
        LanguageWindow.title("Select Language")
        LanguageWindow.geometry("800x500")
        global LanguageFrame
        LanguageFrame=Frame(LanguageWindow)
        Language_Label=Label(LanguageFrame,text="Select Language")
        Language_Label.place(relx = 0.0,
                 rely = 1.0,anchor='n')
        Language_Label.pack()
        
        global options
        options = StringVar(LanguageFrame)
        options.set("English") # default value

        l1 =Label(LanguageFrame, text='Select One', width=10 )  
        l1.pack()

        om1 =OptionMenu(LanguageFrame, options,"English" , "Urdu","Arabic","Turkish", "Chinese", "French", "German")
        om1.pack()

        b1 = Button(LanguageFrame,  text='Extract', command=lambda: my_show(canvas) )  
        b1.pack()

        global str_out
        str_out=StringVar(LanguageFrame)
        str_out.set(" ")

        l2 = Label(LanguageFrame,  textvariable=str_out, width=10 )  
        l2.pack()
        
        global Language_Label2
        Language_Label2=Label(LanguageFrame,text=" ",bd=1, highlightthickness=0.5)
        Language_Label2.place(relx = 0.0,rely = 1.0,anchor='n') 
        Language_Label2.pack()
        
        LanguageFrame.pack()

        canvas.data.undoQueue.append(canvas.data.image.copy())
        canvas.data.imageForTk=makeImageForTk(canvas)
        drawImage(canvas)

        
def my_show(canvas):
    global str_out
    global options
    str_out.set(options.get())
    text=image_to_text(canvas)
    global LanguageFrame
    global Language_Label2
 
        
    if text !=None:
        translator = Translator()
        if (options.get()=="English"):
            language='en'
        if (options.get()=="Urdu"):
            language='ur'
        if (options.get()=="Arabic"):
            language='ar'
        if (options.get()=="Turkish"):
            language='tr'
        if (options.get()=="Chinese"):
            language='zh-cn'
        if (options.get()=="French"):
            language='fr'
        if (options.get()=="German"):
            language='de'
       
        translated_text = translator.translate(text, src='en', dest = language)
        Language_Label2.config(text=translated_text.text)
        
        file2write=open("extract.txt",'w')
        file2write.write(text)
        file2write.close()
    else:
        tkinter.messagebox.showinfo(title="Text Extraction",\
        message="No Text Found" , parent=canvas.data.mainWindow)
            

# Convert Extracted Text in Audio

In [33]:
def text_to_speech(canvas, language):
    image_text=image_to_text(canvas)
    if image_text!= None:
        translator = Translator()
        translated_text = translator.translate(image_text, src='en', dest = language)
        speech = gTTS(translated_text.text, lang = language, slow = False)
        speech.save(language+".mp3")
    
def play(canvas):
    canvas.data.colourPopToHappen=False
    canvas.data.cropPopToHappen=False
    canvas.data.drawOn=False
    if canvas.data.image!=None:
        LanguageWindow=Toplevel(canvas.data.mainWindow)
        LanguageWindow.title("Select Language")
        LanguageWindow.geometry("600x450")
        global LanguageFrame
        LanguageFrame=Frame(LanguageWindow)
        Language_Label=Label(LanguageFrame,text="Select Language")
        Language_Label.place(relx = 0.0,
                 rely = 1.0,anchor='n')
        Language_Label.pack()
        
        global options
        options = StringVar(LanguageFrame)
        options.set("English") # default value

        l1 =Label(LanguageFrame, text='Select One', width=10 )  
        l1.pack()

        om1 =OptionMenu(LanguageFrame, options,"English" ,"Urdu","Arabic","Turkish", "Chinese", "French", "German")
        om1.pack()

        b1 = Button(LanguageFrame,  text='PLAY', command=lambda: myAudio(canvas) )  
        b1.pack()

        global str_out
        str_out=StringVar(LanguageFrame)
        str_out.set(" ")

        l2 = Label(LanguageFrame,  textvariable=str_out, width=10 )  
        l2.pack()

        LanguageFrame.pack()
        canvas.data.undoQueue.append(canvas.data.image.copy())
        canvas.data.imageForTk=makeImageForTk(canvas)
        drawImage(canvas)
        
def myAudio(canvas):
    global str_out
    global options
    str_out.set(options.get())
    text=image_to_text(canvas)     
    if text !=None:
        translator = Translator()
        if (options.get()=="English"):
            language='en'
        if (options.get()=="Urdu"):
            language='ur'
        if (options.get()=="Arabic"):
            language='ar'
        if (options.get()=="Turkish"):
            language='tr'
        if (options.get()=="Chinese"):
            language='zh-cn'
        if (options.get()=="French"):
            language='fr'
        if (options.get()=="German"):
            language='de'

        
        text_to_speech(canvas, language)
        os.system("start "+language+".mp3")
    else:
        tkinter.messagebox.showinfo(title="Audio Player",\
        message="No Text Found" , parent=canvas.data.mainWindow)
            
    


# User Interface

In [34]:

def init(root, canvas):

    buttonsInit(root, canvas)
    menuInit(root, canvas)
  #  labelInit(root,canvas)
    canvas.data.image=None
    canvas.data.textbox=None
    canvas.data.angleSelected=None
    canvas.data.rotateWindowClose=False
    canvas.data.brightnessWindowClose=False
    canvas.data.brightnessLevel=None
    canvas.data.histWindowClose=False
    canvas.data.solarizeWindowClose=False
    canvas.data.posterizeWindowClose=False
    canvas.data.colourPopToHappen=False
    canvas.data.cropPopToHappen=False
    canvas.data.endCrop=False
    canvas.data.drawOn=True
    
    canvas.data.undoQueue=deque([], 10)
    canvas.data.redoQueue=deque([], 10)
    canvas.pack()

    

    
def buttonsInit(root, canvas):
    backgroundColour="white"
    buttonWidth=40
    buttonHeight=2
    toolKitFrame=Frame(root)
    cropButton=Button(toolKitFrame, text="Crop",\
                      background=backgroundColour ,\
                      width=buttonWidth, height=buttonHeight, \
                      command=lambda:crop(canvas))
    cropButton.grid(row=0,column=0)
    rotateButton=Button(toolKitFrame, text="Rotate",\
                        background=backgroundColour, \
                        width=buttonWidth,height=buttonHeight, \
                        command=lambda: rotate(canvas))
    rotateButton.grid(row=1,column=0)
    brightnessButton=Button(toolKitFrame, text="Brightness",\
                            background=backgroundColour ,\
                            width=buttonWidth, height=buttonHeight,\
                            command=lambda: brightness(canvas))
    brightnessButton.grid(row=2,column=0)
    mirrorButton=Button(toolKitFrame, text="Mirror",\
                        background=backgroundColour, \
                        width=buttonWidth,height=buttonHeight, \
                        command=lambda: mirror(canvas))
    mirrorButton.grid(row=3,column=0)
    flipButton=Button(toolKitFrame, text="Flip",\
                      background=backgroundColour ,\
                      width=buttonWidth,height=buttonHeight, \
                      command=lambda: flip(canvas))
    flipButton.grid(row=4,column=0)
    histogramButton=Button(toolKitFrame, text="Scan",\
                           background=backgroundColour ,\
                           width=buttonWidth,height=buttonHeight, \
                           command=lambda: imageScanner(canvas))
    histogramButton.grid(row=5,column=0)
    colourPopButton=Button(toolKitFrame, text="Extract Text",\
                           background=backgroundColour, \
                           width=buttonWidth,height=buttonHeight, \
                           command=lambda: extract_text(canvas))
    colourPopButton.grid(row=6,column=0)
   

    transposeButton=Button(toolKitFrame, text="Convert Extracted Text in Audio",\
                           background=backgroundColour, width=buttonWidth,\
                           height=buttonHeight,command=lambda: play(canvas))
    transposeButton.grid(row=7,column=0)
    
    resetButton=Button(toolKitFrame, text="Reset",\
                       background=backgroundColour ,width=buttonWidth,\
                       height=buttonHeight, command=lambda: reset(canvas))
    resetButton.grid(row=8,column=0)
    
    toolKitFrame.pack(side=LEFT)

def menuInit(root, canvas):
    menubar=Menu(root)
    menubar.add_command(label="New", command=lambda:newImage(canvas))
    menubar.add_command(label="Save", command=lambda:save(canvas))
    menubar.add_command(label="Save As", command=lambda:saveAs(canvas))
    ## Edit pull-down Menu
    editmenu = Menu(menubar, tearoff=0)
    editmenu.add_command(label="Undo   Z", command=lambda:undo(canvas))
    editmenu.add_command(label="Redo   Y", command=lambda:redo(canvas))
    menubar.add_cascade(label="Edit", menu=editmenu)
    root.config(menu=menubar)
    ## Filter pull-down Menu
    filtermenu = Menu(menubar, tearoff=0)
    filtermenu.add_command(label="Invert", \
                           command=lambda:invert(canvas))
    filtermenu.add_command(label="Solarize", \
                           command=lambda:solarize(canvas))
    menubar.add_cascade(label="Filter", menu=filtermenu)
    root.config(menu=menubar)
    


def run():
    # create the root and the canvas
    root = Tk()
    root.title("Image Editor")
    root.iconbitmap("icon.ico")
    canvasWidth=500
    canvasHeight=500
    canvas = Canvas(root, width=canvasWidth, height=canvasHeight, \
                    background="gray")
    # Set up canvas data and call init
    class Struct: pass
    canvas.data = Struct()
    canvas.data.width=canvasWidth
    canvas.data.height=canvasHeight
    canvas.data.mainWindow=root
    init(root, canvas)
    root.bind("<Key>", lambda event:keyPressed(canvas, event))
    # and launch the app
    root.mainloop()  # This call BLOCKS (so your program waits)


run()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Lab1\anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "<ipython-input-34-5578e0a90eb8>", line 95, in <lambda>
    command=lambda:invert(canvas))
  File "<ipython-input-29-c37d86455a2a>", line 6, in invert
    canvas.data.image=ImageOps.invert(canvas.data.image)
  File "C:\Users\Lab1\anaconda3\lib\site-packages\PIL\ImageOps.py", line 525, in invert
    return _lut(image, lut)
  File "C:\Users\Lab1\anaconda3\lib\site-packages\PIL\ImageOps.py", line 58, in _lut
    raise OSError("not supported for this image mode")
OSError: not supported for this image mode
